In [3]:
import polars as pl
import altair as alt

In [6]:
pl.Config.set_tbl_rows(50)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [5]:
df = pl.read_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).with_columns(
    pl.col("text").str.split("(").list.slice(1).list.first().str.split(')').list.first().str.to_lowercase().str.strip_chars().alias('poznamka')
).drop_nulls(
    subset=['datum']
)

In [7]:
len(df)

711719

In [8]:
def zavorka(retezec, frejm=df):
    return frejm.filter(
        (
            pl.col('text').str.contains(retezec) 
            & 
            pl.col('text').str.contains('(',literal=True)
        ) 
        | 
        (
            pl.col('mluvci').str.contains(retezec) 
            & 
            pl.col('mluvci').str.contains('(',literal=True)
        )
).group_by_dynamic(
    index_column='datum',every='1y').agg(pl.col('text').unique().len()
).join(
    frejm.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()), 
    on='datum', how='left'
).with_columns(
    (pl.col('text') / pl.col('text_right')).alias('podil')
).with_columns(
        pl.lit(retezec).alias('co')
)

In [9]:
df.group_by(
    "poznamka"
).len(
).sort(
    by='len',
    descending=True
).head(50)

poznamka,len
str,u32
null,507488
"""děje se.""",12040
"""potlesk.""",11817
"""zvoní""",3650
"""souhlas.""",3305
"""výborně!""",1711
"""ano.""",1545
"""nesouhlasné.""",1486
"""?""",1460


In [103]:
def zavorka(retezec, frejm=df):
    return frejm.filter(
        pl.col('text').str.split('(').list.slice(1).str.split(')').list.slice(0).str.contains(retezec)  
        | 
        pl.col('mluvci').str.split('(').list.slice(1).str.split(')').list.slice(0).str.contains(retezec)
).group_by_dynamic(
    index_column='datum',every='1y').agg(pl.col('text').unique().len()
).join(
    frejm.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()), 
    on='datum', how='left'
).with_columns(
    (pl.col('text') / pl.col('text_right')).alias('podil')
)

In [119]:
alt.Chart(
    zavorka("\([Pp]otlesk").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_7052\1769488776.py:2: SyntaxWarning: invalid escape sequence '\('
  zavorka("\([Pp]otlesk").to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [123]:
alt.Chart(
    zavorka("\([Ss]mích").to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_7052\2024641272.py:2: SyntaxWarning: invalid escape sequence '\('
  zavorka("\([Ss]mích").to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [155]:
alt.Chart(
    zavorka("\([Vv]ýkřiky").to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_7052\3094193495.py:2: SyntaxWarning: invalid escape sequence '\('
  zavorka("\([Vv]ýkřiky").to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [161]:
df.filter(pl.col("text").str.contains("[Bb]učení"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool
"""Ministr zdravotnictví ČR David Rath""","""2002ps_stenprot_055schuz_s055136.htm""",null,"""Děkuji. Je to myslím docela zase signifikantní a typické, když názor vyslovený těmi, kterých se to bytostně týká, tedy lékaři zaměstnanci a zdravotním…",15,"""ps""",2002,55,136,2006-04-21,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""55. schůze""",true,true
"""Poslanec David Rath""","""2006ps_stenprot_027schuz_s027049.htm""",null,"""Dámy a pánové, pane předsedající, já bych prosil, abyste poučil o slušném chování poslance ODS, aby zde nebučeli. Nejsou, jak jsou zvyklí, v chlívě. Z…",3,"""ps""",2006,27,49,2008-01-30,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""27. schůze""",true,true
"""Místopředseda PSP Vojtěch Pikal""","""2017ps_stenprot_040schuz_s040102.htm""",null,"""Děkuji za slovo. Protože nemáme tu britskou tradici, tak tady neprobíhalo během předchozích projevů bučení a podobně, místo toho všeobecný nezájem pří…",7,"""ps""",2017,40,102,2020-01-23,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""40. schůze""",true,true
"""Předsedkyně PSP Markéta Pekarová Adamová""","""2021ps_stenprot_005schuz_s005019.htm""",null,"""Vzhledem k tomu, že program schůze nebyl schválen a nikdo další se s přednostním právem nehlásí, tak já tuto 5. schůzi Poslanecké sněmovny končím. (Po…",2,"""ps""",2021,5,19,2021-12-15,"""PČR, PS 2021-...""","""5. schůze""",true,true
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""",null,"""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true
"""Poslanec Jakub Michálek""","""2021ps_stenprot_055schuz_s055007.htm""",null,"""Děkuji za slovo. Vážená paní předsedkyně, vážené kolegyně, vážení kolegové, pojďme prosím nechat v rozpravě vystoupit i ministry, například tady pana …",9,"""ps""",2021,55,7,2023-02-28,"""PČR, PS 2021-...""","""55. schůze""",true,true


In [157]:
alt.Chart(
    zavorka("\([Bb]učen").to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_7052\4098534515.py:2: SyntaxWarning: invalid escape sequence '\('
  zavorka("\([Bb]učen").to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [159]:
df.filter(pl.col("text").str.contains("(Buče",literal=True))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""",null,"""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true


In [163]:
alt.Chart(
    zavorka("\([Pp]osmě").to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_7052\2904804961.py:2: SyntaxWarning: invalid escape sequence '\('
  zavorka("\([Pp]osmě").to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [149]:
df.filter(
    pl.col("datum").dt.year().is_between(1940,1970)
).filter(
    pl.col("text").str.contains("(Smích",literal=True)
)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool
null,"""1946uns_stenprot_056schuz_s056018.htm""",null,"""Chci zase říci, že naši ministři na tom vinni nejsou. Ministr dr Stránský, ministr dr Dolanský, ministr Nosek jsou ochotni tyto věci udělat, ale oni t…",1,"""uns""",1946,56,18,1947-06-10,"""Ústavodárné NS RČS 1946-1948""","""56. schůze""",true,true
"""Posl. Harus""","""1960ns_stenprot_004schuz_s004023.htm""",null,"""Vážení soudruzi a soudružky poslanci, chci se v první řadě omluvit, neboť jak jste byli svědky, přihlásil jsem se ke slovu teprve nyní, při projednává…",3,"""ns""",1960,4,23,1960-11-17,"""NS RČS 1960-1964""","""4. schůze""",true,true
"""Posl. Fogel""","""1960ns_stenprot_010schuz_s010009.htm""",null,"""Súdružky a súdruhovia poslanci! Vládny návrh zákona o disciplinárnej odpovednosti sudcov z povolania, ktorý dnes prejednávame, je jedným z mnohých opa…",6,"""ns""",1960,10,9,1961-11-29,"""NS RČS 1960-1964""","""10. schůze""",true,true
"""Posl. Borůvka""","""1960ns_stenprot_010schuz_s010018.htm""",null,"""Soudružky a soudruzi poslanci! Plně potvrzuji pravdivost zprávy soudruha Šimůnka o vážné situaci v zemědělské výrobě, a to jak z hlediska vlastních zk…",2,"""ns""",1960,10,18,1961-11-30,"""NS RČS 1960-1964""","""10. schůze""",true,true
"""Zpravodaj posl. Borůvka""","""1960ns_stenprot_013schuz_s013007.htm""",null,"""Vážené Národní shromáždění, soudruzi a soudružky poslanci! Naše sociální a zdravotní zabezpečení dělníků a ostatních zaměstnanců je bezesporu nejlepší…",4,"""ns""",1960,13,7,1962-03-29,"""NS RČS 1960-1964""","""13. schůze""",true,true
null,"""1960ns_stenprot_023schuz_s023016.htm""",null,"""A i já využívám právě teď příležitosti, abych stejně naléhavě připomněl slib, který nám byl dán v pátek 24. ledna t. r. náměstkem předsedy SPK s. Dvoř…",1,"""ns""",1960,23,16,1964-01-30,"""NS RČS 1960-1964""","""23. schůze""",true,true
"""Ministr s. Lomský""","""1964ns_stenprot_008schuz_s008038.htm""",null,"""Vážené soudružky a soudruzi poslanci, s. posl. inž. Gudrich mne interpeloval v otázce uvolnění továrního objektu v jeho volebním obvodě pro rozvoj výr…",2,"""ns""",1964,8,38,1965-11-12,"""NS RČS 1964-1968""","""8. schůze""",true,true
"""Post. Harus""","""1964ns_stenprot_008schuz_s008038.htm""",null,"""K tomu, že jsem poprosil soudruha předsedu o udělení slova po soudruhu ministrovi mě vyprovokoval ministr sám. Právě o této věci jsem s ním včera hovo…",6,"""ns""",1964,8,38,1965-11-12,"""NS RČS 1964-1968""","""8. schůze""",true,true
"""Posl. Kopecká""","""1964ns_stenprot_011schuz_s011033.htm""",null,"""Soudružky a soudruzi, nebojte se, mám jen dotaz. (Smích.) Jako poslanec bych samozřejmě neměla přicházet s požadavkem na pracovní síly a na koruny, al…",2,"""ns""",1964,11,33,1966-06-30,"""NS RČS 1964-1968""","""11. schůze""",true,true


In [111]:
alt.Chart(
    zavorka("potlesk").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [101]:
alt.Chart(
    zavorka("smích").to_pandas(),
    title=["Podíl projevů v NS / FS / PSP, ve kterých","nebo do kterých zazněl podle stenozáznamů smích"],
    width = 300
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None))
)

alt.Chart(...)

In [151]:
alt.Chart(
    pl.concat([zavorka("smích"), zavorka("potlesk"), zavorka("křik")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [153]:
alt.Chart(
    pl.concat([zavorka("\(Smích"), zavorka("\(Potlesk"), zavorka("\(Křik")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_7052\2268805364.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Smích"), zavorka("\(Potlesk"), zavorka("\(Křik")]).to_pandas(),
C:\Users\micha\AppData\Local\Temp\ipykernel_7052\2268805364.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Smích"), zavorka("\(Potlesk"), zavorka("\(Křik")]).to_pandas(),
C:\Users\micha\AppData\Local\Temp\ipykernel_7052\2268805364.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Smích"), zavorka("\(Potlesk"), zavorka("\(Křik")]).to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype para

alt.Chart(...)

In [15]:
alt.Chart(
    pl.concat([zavorka("\(Potlesk zleva"), zavorka("\(Potlesk zprava")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_7696\2118327673.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Potlesk zleva"), zavorka("\(Potlesk zprava")]).to_pandas(),
C:\Users\micha\AppData\Local\Temp\ipykernel_7696\2118327673.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Potlesk zleva"), zavorka("\(Potlesk zprava")]).to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)